for the beginning, the provider have one [beginner guide](https://yichen115.github.io/%E4%B8%8A%E6%89%8B%E6%8C%87%E5%8D%97/PowerShorter%E5%BC%80%E7%AE%B1%E4%B8%8E%E7%AE%80%E5%8D%95%E4%BD%BF%E7%94%A8/), here we will follow the guide to it.

## init the power shorter

the power shorter is based on the serial port, so we need find the serial device first, here we on the linux system, so the no need to install extra drivers, is show on the `/dev/ttyUSB0`.

In [2]:
from power_shorter import *

ps = PowerShorter('/dev/ttyUSB0')

In [27]:
import time

# RELAY2 is electrically connected to the relay
ps.relay(RELAY.RELAY2,1)
time.sleep(1)  # 1 second delay
ps.relay(RELAY.RELAY2,0)

## check our target device

our target device is by the serial protocol to link our host machine on `/dev/ttyACM0`, when we write the 'AES\n' to the serial port, it will return the AES encryption result. 

In [120]:
import serial
import time
import binascii

# Open serial connection to the target device
try:
    target_device = serial.Serial('/dev/ttyACM0', 38400, timeout=1)
    time.sleep(1)  # Allow some time for the connection to establish
    
    # Send the AES command to trigger encryption
    target_device.reset_input_buffer()  # Clear any existing input buffer
    target_device.write(b'AES\n')
    
    # Read the binary response
    binary_response = target_device.read(32)  
    
    if binary_response:
        # Print the raw binary response in hexadecimal
        glitch_text = binascii.hexlify(binary_response).decode('utf-8')
        print(f"Raw response (hex): {glitch_text}")
        
        # Try to interpret as ASCII if possible
        try:
            glitch_text = binary_response.decode('ascii', errors='replace')
            print(f"hex: {glitch_text}")
        except UnicodeDecodeError:
            print("Cannot decode as ASCII text")
    else:
        print("No response received from the device")
    
    # Close the connection when done
    target_device.close()
    
except serial.SerialException as e:
    print(f"Error: {e}")
    print("Make sure the device is connected and the port is correct.")

Raw response (hex): 3533396261333161393838393132613862643863656339333331343737343032
hex: 539ba31a988912a8bd8cec9331477402


## Attack AES on the power shorter

First we to see the attack glitch can be created by the power shorter.

later here we use the `osrtoolkit` package to attack the AES. 

In [145]:
import random
import binascii

# Define correct expected value
correct_text = "539ba31a988912a8bd8cec9331477402"

glitch_pulse = random.randint(8, 9)
glitch_delay = random.randint(435600, 442400)
ps.engine_cfg(Engine.E2,[(0,glitch_delay),(1,glitch_pulse),(0,1)])
ps.arm(Engine.E2)

time.sleep(0.5) 
target_device = serial.Serial('/dev/ttyACM0', 38400, timeout=1)
target_device.reset_input_buffer()  # Clear any existing input buffer
target_device.write(b'AES\n')
binary_response = target_device.read(32)


if binary_response:
    try:
        
        # Also decode as ASCII for display purposes
        glitch_text = binary_response.decode('ascii', errors='replace')
        
        state = ps.state(Engine.E2)
        if state == "glitched":
            if glitch_text == correct_text:
                print(f"Glitch PASS: {glitch_text}")
            else:
                print(f"Glitch success: {glitch_text}")
        else:
            print(f"No glitch detected: {glitch_text}")
    except UnicodeDecodeError:
        print("Cannot decode as ASCII text")

Glitch PASS: 539ba31a988912a8bd8cec9331477402
